In [1]:
!pip install --upgrade openai

In [6]:
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import pandas as pd

In [7]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Humza\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [67]:
pattern = r"\b\w+(?:[‘’'-]\w+|\w+'\w+)?\b"
parse_tokenizer = RegexpTokenizer(pattern)

In [68]:
with open('ALL_eminem.txt', 'r', encoding='utf8') as f:
    
    lines = f.readlines()

    data = []

    for line in lines:

        if len(line) == 0:
            continue

        if '[' in line or ']' in line:
            continue

        words = parse_tokenizer.tokenize(line)
        words_tags = nltk.pos_tag(words)
        tags = [i[1] for i in words_tags]

        if len(words) == 0 or len(tags) == 0:
            continue



        # data.append('Fill: ' + (' '.join(tags), ' '.join(words)))
        data.append((tags, words))

In [64]:
i = 1
sonnets = []
tmp = []
tmp_tags = []
tmp_words = []
for tags, words in data[:30]:

    if i % 4 != 0:
        tmp_tags.append(' '.join(tags))
        tmp_words.append(' '.join(words))
        
    if i % 4 == 0:
        sonnets.append((tmp_tags, tmp_words))
        tmp_tags = []
        tmp_words = []
        
    # print(tags, words)

In [65]:
print(sonnets)

[(['NN PRP VBD VBN VBP JJ IN PRP RB TO VB PRP$ NNS', 'CC NNP RB VBG TO VB DT CD NN', 'CD NNS CD NNS'], ['Look I was gonna go easy on you not to hurt your feelings', "But I'm only going to get this one chance", 'Six minutes six minutes']), (['CD NNS CD NNS NNP NNP NN IN', 'RB DT NN NNP VBD', 'IN NN IN TO VB'], ["Six minutes six minutes Slim Shady you're on", "Just a feeling I've got", "Like something's about to happen"]), (['IN DT VBZ WP PRP VBP PRP VBZ NN IN NN', 'NNP NN CC IN PRP VBZ RB JJ IN PRP VBP', 'NNP RB VBG DT NNS'], ["If that means what I think it means we're in trouble", 'Big trouble And if he is as bananas as you say', "I'm not taking any chances"]), (['NNP VBG TO VB IN DT NNP NNP NNP NNP', 'DT PRP$ NNS IN DT NN TO DT NN NN RB NN', 'RB WP VBP PRP$ NNS VBP RB RB TO VB NN NN NN'], ["I'm beginning to feel like a Rap God Rap God", 'All my people from the front to the back nod back nod', 'Now who thinks their arms are long enough to slap box slap box']), (['CC IN PRP TO VB IN DT 

In [135]:
group_size = 14
num_groups = len(data) // group_size
groups = [data[i*group_size:(i+1)*group_size] for i in range(num_groups)]
# groups = [data[i:i+group_size] for i in range(0, len(data), group_size)]
sonnet_form = []
for group in groups:
    pos_tags = ''
    lyrics = ''
    for i in group:
        if i[0] == group[-1][0]:
            # pos_tags.append(' '.join(i[0]) + ' ->')
            pos_tags = pos_tags + ' '.join(i[0])
            lyrics = lyrics + ' '.join(i[1])
        else:             
            pos_tags = pos_tags + ' '.join(i[0]) + '\n'
            lyrics = lyrics + ' '.join(i[1]) + '\n'
            
    sonnet_form.append((pos_tags, lyrics))

In [137]:
sonnet_form[0]

('NN PRP VBD VBN VBP JJ IN PRP RB TO VB PRP$ NNS\nCC NNP RB VBG TO VB DT CD NN\nCD NNS CD NNS\nNNP JJ PRP MD VB PRP\nCD NNS CD NNS NNP NNP NN IN\nRB DT NN NNP VBD\nIN NN IN TO VB\nCC PRP VBP VB WP\nIN DT VBZ WP PRP VBP PRP VBZ NN IN NN\nNNP NN CC IN PRP VBZ RB JJ IN PRP VBP\nNNP RB VBG DT NNS\nPRP VBP RB WP DT NN VBD\nNNP VBG TO VB IN DT NNP NNP NNP NNP\nDT PRP$ NNS IN DT NN TO DT NN NN RB NN',
 "Look I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance\nSix minutes six minutes\nSomething's wrong I can feel it\nSix minutes six minutes Slim Shady you're on\nJust a feeling I've got\nLike something's about to happen\nBut I don't know what\nIf that means what I think it means we're in trouble\nBig trouble And if he is as bananas as you say\nI'm not taking any chances\nYou are just what the doc ordered\nI'm beginning to feel like a Rap God Rap God\nAll my people from the front to the back nod back nod")

In [138]:
df = pd.DataFrame.from_records(sonnet_form, columns=['prompt', 'completion'])

In [139]:
df.head()

,prompt,completion
0,NN PRP VBD VBN VBP JJ IN PRP RB TO VB PRP$ NNS...,Look I was gonna go easy on you not to hurt yo...
1,RB WP VBP PRP$ NNS VBP RB RB TO VB NN NN NN\nP...,Now who thinks their arms are long enough to s...
2,IN DT NN NN\nVBG DT NNP IN DT NN IN DT NNP\nNN...,With this rappity brat\nPacking a MAC in the b...
3,IN DT NN IN NNP RB VBG DT NN NN\nNNP NNP VBG T...,For the wack while I'm masterfully constructin...
4,RB PRP VBP DT JJ NN JJ IN NNS\nNNP NNP DT NN I...,Now I lead a new school full of students\nMe I...


In [140]:
df.to_json("eminem_sonnet_parsed.jsonl", orient='records', lines=True)

In [141]:
sum = 0
for line in df['completion']:
    sum += len(line.split(' '))
sum

157777

In [142]:
sum = 0
for line in df['prompt']:
    # print(line)
    sum += len(line.split(' '))
sum

157777

In [131]:
df = pd.read_json('eminem_sonnet_parsed.jsonl', lines=True)

In [132]:
!openai tools fine_tunes.prepare_data -f eminem_sonnet_parsed.jsonl

^C


In [143]:
import json
from sklearn.model_selection import train_test_split

with open('eminem_sonnet_parsed_prepared.jsonl', 'r', encoding='utf8') as f:
    data = [json.loads(line) for line in f]

train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

with open('eminem_sonnet_parsed_train.jsonl', 'w') as f:
    for example in train_data:
        f.write(json.dumps(example) + '\n')

with open('eminem_sonnet_parsed_val.jsonl', 'w') as f:
    for example in val_data:
        f.write(json.dumps(example) + '\n')

In [146]:
import os
import openai
os.environ['OPENAI_API_KEY'] = 'INSERT OPENAI API KEY HERE'

In [150]:
openai.File.create(
  file=open("eminem_sonnet_parsed_train.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-LmGDRsSyz1OcNysYd84giRQ5 at 0x24de2e64cc0> JSON: {
  "bytes": 1426217,
  "created_at": 1682993457,
  "filename": "file",
  "id": "file-LmGDRsSyz1OcNysYd84giRQ5",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [149]:
openai.File.create(
  file=open("eminem_sonnet_parsed_val.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-0vfhb8CzRJkNMZ80vO6LRCAt at 0x24de2cd1b80> JSON: {
  "bytes": 160914,
  "created_at": 1682993418,
  "filename": "file",
  "id": "file-0vfhb8CzRJkNMZ80vO6LRCAt",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [151]:
for x in openai.File.list()['data']:
    print(x['filename'], x['status'])

compiled_results.csv processed
compiled_results.csv processed
compiled_results.csv processed
file processed
file uploaded
hs_cs_train.jsonl processed
hs_cs_prepared.jsonl processed
compiled_results.csv processed
eminem_lyrics_parsed_val.jsonl processed
eminem_lyrics_parsed_train.jsonl processed
compiled_results.csv processed


In [152]:
# !openai api fine_tunes.create -t "eminem_lyrics_parsed_train.jsonl" -v "eminem_lyrics_parsed_val.jsonl" -m curie --suffix "eminem_v3"
openai.FineTune.create(training_file="file-LmGDRsSyz1OcNysYd84giRQ5", validation_file="file-0vfhb8CzRJkNMZ80vO6LRCAt", model="curie", suffix="eminem_sonnet")
#ft-bBtFwyCShN6aeUkjZuOiFJy9

<FineTune fine-tune id=ft-TVUIK91MTMejl3QTJKd7qKgA at 0x24deccc9590> JSON: {
  "created_at": 1682993495,
  "events": [
    {
      "created_at": 1682993495,
      "level": "info",
      "message": "Created fine-tune: ft-TVUIK91MTMejl3QTJKd7qKgA",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-TVUIK91MTMejl3QTJKd7qKgA",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-X0kzGQIW73wa0UvE4Oz6MEyu",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 1426217,
      "created_at": 1682993457,
      "filename": "file",
      "id": "file-LmGDRsSyz1OcNysYd84giRQ5",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1682993495,
  "validation_files": [
    {
      "bytes": 160914,
   

In [94]:
# !openai api fine_tunes.follow -i "ft-i03tSuNxp7rvVeiOrxmoAzZd"
openai.FineTune.retrieve(id="ft-i03tSuNxp7rvVeiOrxmoAzZd")

<FineTune fine-tune id=ft-i03tSuNxp7rvVeiOrxmoAzZd at 0x266863469a0> JSON: {
  "created_at": 1682395983,
  "events": [
    {
      "created_at": 1682395983,
      "level": "info",
      "message": "Created fine-tune: ft-i03tSuNxp7rvVeiOrxmoAzZd",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-i03tSuNxp7rvVeiOrxmoAzZd",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-X0kzGQIW73wa0UvE4Oz6MEyu",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 1777966,
      "created_at": 1682395929,
      "filename": "eminem_lyrics_parsed_train.jsonl",
      "id": "file-4CampKIBbbj8TBSeCKyW49X5",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1682395983,
  "validation_files": [
    {


In [154]:
openai.FineTune.list_events(id='ft-TVUIK91MTMejl3QTJKd7qKgA', stream=True)

KeyboardInterrupt: 

In [ ]:
openai.FineTune.retrieve(id='ft-TVUIK91MTMejl3QTJKd7qKgA')

In [56]:
import openai

In [70]:
!openai api fine_tunes.follow -i "ft-DyMnU1ARWFSAS2dj90u9S0rs"

^C
[2023-04-24 22:58:51] Created fine-tune: ft-DyMnU1ARWFSAS2dj90u9S0rs
[2023-04-24 22:58:56] Fine-tune costs $0.72
[2023-04-24 22:58:56] Fine-tune enqueued. Queue number: 0
[2023-04-24 22:59:00] Fine-tune started

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-DyMnU1ARWFSAS2dj90u9S0rs



In [ ]:
#ft-T9cjhrbFJDq2WTQBKBlwtnsQ"

In [95]:
!openai api fine_tunes.get -i "ft-i03tSuNxp7rvVeiOrxmoAzZd"

{
  "created_at": 1682395983,
  "events": [
    {
      "created_at": 1682395983,
      "level": "info",
      "message": "Created fine-tune: ft-i03tSuNxp7rvVeiOrxmoAzZd",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-i03tSuNxp7rvVeiOrxmoAzZd",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-X0kzGQIW73wa0UvE4Oz6MEyu",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 1777966,
      "created_at": 1682395929,
      "filename": "eminem_lyrics_parsed_train.jsonl",
      "id": "file-4CampKIBbbj8TBSeCKyW49X5",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1682395983,
  "validation_files": [
    {
      "bytes": 197223,
      "created_at": 1682395931,
      "filename": "e

In [59]:
!openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1682380810,
      "fine_tuned_model": "ada:ft-personal:hatespeech-counterspeech-2023-04-25-00-13-31",
      "hyperparams": {
        "batch_size": 8,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-bj9mfD3ofGuxY6DGAk7E6bVt",
      "model": "ada",
      "object": "fine-tune",
      "organization_id": "org-X0kzGQIW73wa0UvE4Oz6MEyu",
      "result_files": [
        {
          "bytes": 116611,
          "created_at": 1682381611,
          "filename": "compiled_results.csv",
          "id": "file-cF0EG0XKY9H8n8uufhqQYkEZ",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 1019363,
          "created_at": 1682380810,
          "filename": "hs_cs_train.jsonl",
          "id": "file-rZ0lJZbusA

In [ ]:
# ft-1Mamzo5GHF2maepkr019CY86 succeeded

In [ ]:
prompts = ['PRP VBZ IN DT NN IN DT JJ NN IN DT NN ->',
'WP VBZ PRP RB IN NN DT JJ NN IN DT NN ->',
'PRP$ NNS VBP DT NN CC DT JJ NN IN DT NN ->',
'PRP VBP IN DT NN CC DT JJ NN IN DT NN ->',
'DT NNS WDT VBP IN DT NNS IN NN IN DT NN ->',
'TO VB TO VB RB JJ CC JJ RB IN NN MD ->',
'CC IN NN PRP IN NN IN PRP$ NNS IN DT NN ->',
'PRP$ JJ NN NN PRP VBP DT JJ NN IN DT NN ->',
'VB PRP NNP RB MD PRP VB PRP$ NN IN DT NN ->',
'IN PDT DT NN PRP VBP DT JJ NN IN DT NN ->',
'NNP NN JJ NN CC NN DT JJ NN IN DT NN ->',
'VBG DT PRP$ NNS IN NN IN DT NN DT JJ NN ->',
'CC RB DT JJ NN IN DT JJ NN IN DT NN ->']